In [110]:
import pennylane as qml
import numpy as np
import torch
from torch import optim
from collections import defaultdict
import networkx as nx
import copy
import math
from itertools import combinations
import sys
from pennylane_ionq import SimulatorDevice
API_KEY = "1M2CfwlG0uQPCJCNulP2rGYd9JTlXLbS"

In [111]:
class Chess:
  def __init__(self, size = 4, horses = []):
    '''
      Args :
        size(int) : size of Chess Board
        horses(list[list[int]]) : indexes(row, col) of placed horses, Not Duplicated with Same Row or Column
    '''
    is_nonempty = defaultdict(int)
    self.board = [[0 for _ in range(size)] for _ in range(size)]
    self.horses = horses
    self.size = size
    for h in horses:
      self.board[h[0]][h[1]] = 1
      is_nonempty[h[0]] += 1
    self.empty_rows = []
    self.nonempty_rows = []
    for i in range(size):
      if is_nonempty[i] == 1:
        self.nonempty_rows.append(i)
      else:
        self.empty_rows.append(i)

  def set_horses(self, new_horses):
    self.horses = self.horses + new_horses
    horses = self.horses
    size = self.size
    is_nonempty = defaultdict(int)
    for h in horses:
      self.board[h[0]][h[1]] = 1
      is_nonempty[h[0]] += 1
    self.empty_rows = []
    self.nonempty_rows = []
    for i in range(size):
      if is_nonempty[i] == 1:
        self.nonempty_rows.append(i)
      else:
        self.empty_rows.append(i)



  def draw(self):
    boxes = ['□', '◆']
    for row_board in self.board:
      for b in row_board:
        print(boxes[b], end ='')
      print()

In [112]:
C1 = Chess(4, [[0, 1]]) 

In [136]:
# 필요 변수값 설정
board_size = C1.size
empty_rows = C1.empty_rows
required_isingXY = board_size * len(empty_rows)
num_of_layers = 4

#dev = SimulatorDevice(wires= board_size * board_size,shots=10000,api_key=API_KEY) #simulation device 설정
dev = qml.device("defait.qubit",wires= board_size * board_size) #device 설정


wires= board_size * board_size
w = torch.tensor(torch.rand(num_of_layers * required_isingXY) * 2 * np.pi, requires_grad = True)
N_iter = 1000

C:\Users\wlsgu\AppData\Local\Temp\ipykernel_12008\3352730697.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w = torch.tensor(torch.rand(num_of_layers * required_isingXY) * 2 * np.pi, requires_grad = True)


In [114]:
C1 = Chess(4, [[0, 1]])
A = np.array(C1.board).T
B = np.flip(A)
ham = 0
n = C1.size
for i in range(n):
    ham += pow(np.sum(A[i])-1,2)/n
    
for i in range(2*n-3):
    diag1 = np.diag(A,i-(n-2))
    diag2 = np.diag(B,i-(n-2))
    ham += np.sum(diag1)*(np.sum(diag1)-1)/(2*n-3)
    ham += np.sum(diag2)*(np.sum(diag2)-1)/(2*n-3)
print(ham)

0.75


In [117]:
def cost_f(new_board,C = C1):
    
  board = torch.tensor(new_board).numpy().reshape(4,4)
  ham = 0
  n = C.size
  A = np.array(board).T
  B = np.flip(A)
  B = B[::-1]

  # 세로 확인 
  for i in range(n):
    ham += pow(np.sum(A[i])-1,2)/n
    
  # 대각 확인
  for i in range(2*n-3):
    diag1 = np.diag(A,i-(n-2))
    diag2 = np.diag(B,i-(n-2))
    ham += np.sum(diag1)*(np.sum(diag1)-1)/(2*n-3)
    ham += np.sum(diag2)*(np.sum(diag2)-1)/(2*n-3)
  return torch.tensor(ham,requires_grad=True)

In [118]:
#VQA Layer building
def vqa_layer(thetas = w, size = board_size, empty_rows = empty_rows, n_layer = num_of_layers): #Ansatz
    th_count = 0
    n = board_size
    for i in range(n_layer):
        for e in empty_rows:
            pivot = n * e # qubit 번호 구하기, board의 가로세로 사이즈 * row 번호
            for j in range(n):
                qml.IsingXY(thetas[th_count], wires = [pivot + ( (0 - j) % n ), pivot + ( (0 - j - 1) % n )])
                th_count += 1

In [119]:
@qml.qnode(dev, interface='torch')
def main_circuit(thetas = w, board = C1, n_layer = num_of_layers):
  n = C1.size
  empty_rows = C1.empty_rows
  horses = C1.horses
  for h in horses:
    qml.PauliX(h[0] * n + h[1])
  for e in empty_rows:
    qml.PauliX(e * n)
  vqa_layer() #Ansatz Apply
  
  # return qml.probs()
  new_board = []
  for i in range(n*n):
      new_board.append(qml.expval(qml.PauliZ(i)))
  return new_board
  pass

In [137]:
optimizer = optim.SGD([w],lr=0.05,momentum=1) #lr : learning rate, SGD : gradient descent
for iter in range(N_iter):
    #cost function
    cost = cost_f(main_circuit(w,C1,num_of_layers),C1)
    #Optimizing step
    optimizer.zero_grad() #gradient 초기화
    cost.backward() #gradient 계산
    optimizer.step() #함수 개선

In [138]:
@qml.qnode(dev, interface='torch')
def state_vec(thetas = w, board = C1, n_layer = num_of_layers):
  n = C1.size
  empty_rows = C1.empty_rows
  horses = C1.horses
  for h in horses:
    qml.PauliX(h[0] * n + h[1])
  for e in empty_rows:
    qml.PauliX(e * n)
  vqa_layer()
  return qml.state()

In [139]:
expect_E = main_circuit(w,C1,num_of_layers)
expect_E = np.sum(torch.tensor(expect_E).numpy())
state_f = state_vec()
ans=0
result = []
print("expectation value of energy : ",expect_E)

c:\Users\wlsgu\AppData\Local\Programs\Python\Python312\Lib\site-packages\pennylane\_qubit_device.py:653: UserWarning: Requested state or density matrix with finite shots; the returned state information is analytic and is unaffected by sampling. To silence this warning, set shots=None on the device.
  warnings.warn(


QuantumFunctionError: The current device is not capable of returning the state

In [ ]:
'''
for i in range(2 ** (board_size * board_size)):
    print(f"probability of getting |{i}> :",abs(state_f[i])**2)
'''
for i in range(2 ** (board_size * board_size)):
    if abs(state_f[i]) ** 2 > 0.1:
        result.append((bin(i)[2:], abs(state_f[i].item())**2))
        print(format(i,'b')+"is ground state with probability",abs(state_f[i].item())**2)

100010000010100is ground state with probability 0.18278855743951972
100010000100100is ground state with probability 0.3975903214095142
100010001000100is ground state with probability 0.27334536563558004


In [ ]:
result.sort(key = lambda x:x[1])
result = result [::-1]
print(result)

[('100010000100100', 0.3975903214095142), ('100010001000100', 0.27334536563558004), ('100010000010100', 0.18278855743951972)]


In [ ]:
def answer(result : dict):
    n = C1.size
    for e in result:
      print('===========================')
      ans = int(e[0])
      coordinates = []
      cnt = n * n - 1
      while cnt >= 0:
        if ans % 10 == 1:
          coordinates.append([cnt // n, cnt % n])
        cnt -= 1
        ans //= 10
      tempC = Chess(n, coordinates)
      tempC.draw()
      print('===========================')

    '''
    for ans in result:
      temp_board  = [[0 for _ in range(n) for _ in range(n)]]
      cnt = n ** 2 - 1
      while cnt > 0:
        i, j = cnt // n, cnt % n
        temp_board[i][j] = ans % 10
        ans //= 10
        cnt -= 1
    '''
    return

answer(result)

□◆□□
□◆□□
□□◆□
□◆□□
□◆□□
□◆□□
□◆□□
□◆□□
□◆□□
□◆□□
□□□◆
□◆□□
